
## 1. 問題の分析

### 競技プログラミング視点での分析

* **実行速度最優先**:
  末尾（LSB）から **Two-Pointer + Carry** で1パス走査。`reverse()`や`unshift()`を避け、**固定長バッファに末尾から書き込む**ことで、割り当て・コピー回数を最小化。
* **メモリ最小化**:
  出力以外の追加メモリは定数（`carry`/インデックス）に限定。**O(1) 補助空間**。

### 業務開発視点での分析

* **型安全性・保守性**:
  関数は **Pure**（副作用なし）。入力検証を分離した軽量ガードで、読みやすさとホットパスの速度を両立。JSDocで仕様を明確化。
* **エラーハンドリング**:
  Type/Rangeの基本検証（文字列・長さ・先頭ゼロ・`'0'|'1'`検査）。LeetCodeでは制約が保証されるため、必要に応じて削除可能。

### TypeScript特有の考慮点

* **型推論**: ループ変数・数値演算は`number`で単型化し、JIT最適化に優しい形に。
* **コンパイル時最適化**: `const`/`readonly`でイミュータブル意識、実行時の型分岐を最小化。
* **null安全性**: インデックス境界での`i>=0`/`j>=0`ガードにより`charCodeAt`の未定義アクセスを回避。

---

## 2. アルゴリズムアプローチ比較

| アプローチ                              | 時間計算量 | 空間計算量 | TS実装コスト | 型安全性 | 可読性 | 備考                   |
| ---------------------------------- | ----- | ----- | ------- | ---- | --- | -------------------- |
| **方法A: Two-Pointer + 固定長バッファ（採用）** | O(n)  | O(1)  | 低       | 高    | 高   | `reverse()`不要、末尾書き込み |
| 方法B: `BigInt`に変換して加算               | O(n)  | O(n)  | 中       | 中    | 中   | 桁数1e4で非推奨、環境依存       |
| 方法C: `push`して`reverse()`で結合        | O(n)  | O(n)  | 低       | 高    | 中   | 可だがメモリ・コピー増          |
---
> `n = max(|a|, |b|)`。

---

## 3. 選択したアルゴリズムと理由

* **選択したアプローチ**: 方法A（Two-Pointer + 固定長バッファ）
* **理由**:

  * **計算量**: 時間 O(n) / 空間 O(1)（出力除く）で最小クラス。
  * **TypeScriptでの型安全性**: 文字→数値を`charCodeAt`基準で明確化し、未定義アクセスを境界ガードで排除。
  * **保守性**: 入力検証は独立・簡潔、コア処理は短く読みやすい。
* **TypeScript特有の最適化ポイント**:

  * `const ZERO = 48` 等の定数化でJIT分岐削減。
  * ループ本体は**単純な数値演算**のみ（`& 1`/`>> 1`）、分岐は境界と`carry`のみに限定。

---

## 4. 実装コード（**LeetCode形式 / ESM**）

```typescript
/**
 * Add Binary — 2進文字列 a, b の和を 2進文字列で返す（Pure）
 * ESM / LeetCode 互換
 *
 * @param a - 2進文字列（'0'/'1' のみ, "0" 以外は先頭ゼロ禁止）
 * @param b - 2進文字列（同上）
 * @returns a + b の2進表現
 * @throws {TypeError} 入力が文字列でない
 * @throws {RangeError} 長さ・文字集合・先頭ゼロの制約違反
 * @complexity Time: O(n), Space: O(1)（出力除く）
 */
function addBinary(a: string, b: string): string {
  // ===== 入力検証（必要に応じて削除可：LeetCodeは制約保証） =====
  if (typeof a !== 'string' || typeof b !== 'string') {
    throw new TypeError('Inputs must be strings.');
  }
  const la = a.length;
  const lb = b.length;
  if (la < 1 || lb < 1 || la > 10_000 || lb > 10_000) {
    throw new RangeError('Input length out of bounds (1..10000).');
  }
  // 先頭ゼロ（"0" を除く）チェック
  if ((la > 1 && a.charCodeAt(0) === 48) || (lb > 1 && b.charCodeAt(0) === 48)) {
    throw new RangeError('Leading zeros are not allowed (except "0").');
  }
  // '0' / '1' のみ
  for (let i = 0; i < la; i++) {
    const c = a.charCodeAt(i);
    if (c !== 48 && c !== 49) throw new RangeError('Input must be binary.');
  }
  for (let i = 0; i < lb; i++) {
    const c = b.charCodeAt(i);
    if (c !== 48 && c !== 49) throw new RangeError('Input must be binary.');
  }
  // ===== 本処理：Two-Pointer + 固定長バッファ =====
  const ZERO = 48; // '0'
  let i = la - 1;
  let j = lb - 1;
  let k = (la > lb ? la : lb); // 書き込みインデックス（末尾）
  let carry = 0;

  // 先頭に1桁ぶん余地（最大で繰り上がりが生じる）
  const out: string[] = new Array(k + 1);

  while (i >= 0 || j >= 0 || carry !== 0) {
    // 範囲外は '0' を装填し、最下位ビットで 0/1 を得る
    const ai = (i >= 0 ? a.charCodeAt(i) : ZERO) & 1;
    const bj = (j >= 0 ? b.charCodeAt(j) : ZERO) & 1;

    const sum = ai + bj + carry; // 0..3
    out[k] = String.fromCharCode(ZERO + (sum & 1)); // '0' or '1'
    carry = sum >> 1;

    i--; j--; k--;
  }

  // k が指す位置より後ろが完成文字列
  return k >= 0 ? out.slice(k + 1).join('') : out.join('');
}

// ESM: ローカル実行や取り込み用（LeetCode側は無視されても可）
export default addBinary;

Analyze Complexity
Runtime 1 ms
Beats 73.30%
Memory 58.67 MB
Beats 11.37%

```

---

## TypeScript固有の最適化観点

* **型安全性の活用**

  * 引数は厳密に`string`。未定義アクセスは`i>=0`/`j>=0`ガードで回避。
  * ループ内での値は**number単型**を維持し、JITに優しい形へ。
* **コンパイル時最適化**

  * `const ZERO` など**定数化**で分岐最小化。
  * 配列`out`は**固定長**で作成し、**末尾から上書き**して**`reverse()`不要**。
* **開発効率と保守性**

  * Pure function + JSDocにより、仕様・制約が明瞭。
  * 入力検証ブロックは**容易に無効化**でき、競技と業務の両要件に適応。

**メモリ削減と微速化**の両面で効きやすいのは、**可変長配列＋`join`をやめて固定長バッファ（`Buffer` or `Uint8Array`）に“末尾から数値（ASCIIコード）を書き込む”**方式です。
これで **中間文字列/配列の生成をゼロ**にでき、ピークメモリ（特に `Array<string>` + `join` 分）が下がります。ループ内で `String.fromCharCode` も不要になり、分配器のオーバーヘッドも減ります。

---

## どこを削るか（要点）

* `reverse()`/`join()`/`String.fromCharCode` の**都度生成を排除**
  → **固定長 `Buffer`** に **ASCIIコード(48/49)** を直接書き込み、最後に `toString('ascii', start)`。
* **ビット値の取得は分岐レス**に：`(charCodeAt(i) & 1)`（'0'=48→0, '1'=49→1）
  範囲外は `0` を与えるだけでOK。
* **入力検証ブロックは任意**：LeetCodeは制約保証のため、速度/メモリを詰めるなら外してよい（下は“高速版”）。

---

## 改善版（TypeScript / ESM / LeetCode関数形式：**高速・低メモリ**）

```typescript
/**
 * Add Binary — 高速・低メモリ Node最適化版（ESM / Pure）
 * @param a 2進文字列
 * @param b 2進文字列
 * @returns a+b の2進表現
 * @complexity Time: O(n), Space: O(1) aux（出力除く）
 */
function addBinary(a: string, b: string): string {
  // --- 制約保証前提の高速版：入力検証は省略（必要なら前処理で追加） ---
  let i = a.length - 1;
  let j = b.length - 1;
  let k = (i > j ? i : j) + 1;      // 書き込み先の末尾（+1 は最上位の繰り上がり分）
  let carry = 0;

  // Node.js v22 なら Buffer が最も軽量（ASCIIに直で書く）
  // 先頭に1桁の余地（最大繰り上がり）を確保
  const buf = Buffer.allocUnsafe(k + 1); // 初期化不要なので Unsafe

  // '0'のASCIIコード
  const ZERO = 48;

  while (i >= 0 || j >= 0 || carry) {
    // 範囲外は 0、ビットは &1 で取得（'0'=48→0, '1'=49→1）
    const ai = i >= 0 ? (a.charCodeAt(i) & 1) : 0;
    const bj = j >= 0 ? (b.charCodeAt(j) & 1) : 0;

    const sum = ai + bj + carry;      // 0..3
    buf[k] = ZERO + (sum & 1);        // '0' or '1' をASCIIで直書き
    carry = sum >> 1;                 // 0 or 1

    i--; j--; k--;
  }

  // k が指す位置より後ろが有効文字列
  // 先頭にダミー'0'が残っていれば skip される
  const start = k + 1;
  return buf.toString('ascii', start);
}

export default addBinary;

Analyze Complexity
Runtime 2 ms
Beats 52.79%
Memory 56.14 MB
Beats 81.83%

```

### ポータブル版（Node以外/ブラウザ想定なら）

`Uint8Array` + `TextDecoder`（UTF-8）でも同様にメモリが削れます。

```typescript
function addBinary(a: string, b: string): string {
  let i = a.length - 1, j = b.length - 1, k = (i > j ? i : j) + 1, carry = 0;
  const out = new Uint8Array(k + 1);  // ASCIIを書き込む
  const ZERO = 48;
  while (i >= 0 || j >= 0 || carry) {
    const ai = i >= 0 ? (a.charCodeAt(i) & 1) : 0;
    const bj = j >= 0 ? (b.charCodeAt(j) & 1) : 0;
    const sum = ai + bj + carry;
    out[k] = ZERO + (sum & 1);
    carry = sum >> 1;
    i--; j--; k--;
  }
  // UTF-8 でも '0'/'1' は同値
  return new TextDecoder().decode(out.subarray(k + 1));
}
export default addBinary;

Analyze Complexity
Runtime 1 ms
Beats 73.30%
Memory 57.62 MB
Beats 53.50%

```

---

## 効果の見込み

* **Memory**: `Array<string>` と `join()` を排除し、**ピークメモリが縮小**。ビート率（~11%→改善）を狙えます。
* **Runtime**: ループ内での**文字列生成ゼロ**化と **固定長バッファ**により、1ms帯でも**安定して上振れ**しやすいです。

> さらに詰めるなら：
>
> * **入力検証を完全オフ**（競技用）
> * 早期return（`a==="0"||b==="0"`）は効果小だが悪化はしません。

この置き換えで、特にメモリ面のビート率向上が期待できます。
